In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import sys

sys.path.append("/home/td-user/rtrec")

In [69]:
from rtrec.experiments.datasets import load_movielens

df = load_movielens(dataset_scale='1m', load_user_attributes=True, load_item_attributes=True)

Using existing ratings.dat file.


In [70]:
from rtrec.experiments.split import temporal_user_split
train_df, test_df = temporal_user_split(df)

# With user/item features

In [71]:
user_tags = train_df.loc[train_df.groupby('user')['tstamp'].idxmax()][['user', 'gender', 'age', 'occupation', 'zip_code']].set_index('user').apply(
    lambda row: {f"gender#{row['gender']}", f"age#{row['age']}", f"occupation#{row['occupation']}", f"zipcode#{row['zip_code']}"},
    axis=1
).to_dict()

In [72]:
item_tags = (
    train_df.groupby("item")["genres"]
    .apply(lambda genres: set(f"genre#{tag}" for genre_list in genres for tag in genre_list.split('|')))
    .to_dict()
)

In [73]:
from rtrec.recommender import Recommender
from rtrec.models import LightFM

model = LightFM(no_components=10, loss="warp", epochs=20, item_alpha=0, user_alpha=0)
recommender = Recommender(model)

In [74]:
recommender.bulk_fit(train_df, user_tags=user_tags, item_tags=item_tags)

Register user features:   0%|          | 0/6040 [00:00<?, ?it/s]


NameError: name 'bisect_left' is not defined

In [45]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)

100%|██████████| 61/61 [00:01<00:00, 50.89it/s]


{'precision': 0.12948675496689188,
 'recall': 0.05974662783556531,
 'f1': 0.06768103547513246,
 'ndcg': 0.1424585923224094,
 'hit_rate': 0.5869205298013245,
 'mrr': 0.28460258330705224,
 'map': 0.07115220559188869,
 'tp': 7821,
 'auc': 0.3257905169242095}

In [46]:
test_user_tags = test_df.loc[test_df.groupby('user')['tstamp'].idxmax()][['user', 'gender', 'age', 'occupation', 'zip_code']].set_index('user').apply(
    lambda row: {f"gender#{row['gender']}", f"age#{row['age']}", f"occupation#{row['occupation']}", f"zipcode#{row['zip_code']}"},
    axis=1
).to_dict()

In [47]:
recommender.evaluate(test_df, user_tags=user_tags, recommend_size=10, filter_interacted=True)

100%|██████████| 61/61 [23:26<00:00, 23.06s/it]    


{'precision': 0.12521523178808316,
 'recall': 0.05411691934082677,
 'f1': 0.06254158825497853,
 'ndcg': 0.13621627077049822,
 'hit_rate': 0.562748344370861,
 'mrr': 0.2687613003258695,
 'map': 0.06833874981958692,
 'tp': 7563,
 'auc': 0.30493160674865966}

In [14]:
users = test_df['user'].unique()

In [15]:
for u in users:
    assert user_tags.pop(u) != None

# Without user/item features

In [ ]:
from rtrec.recommender import Recommender
from rtrec.models import LightFM

model = LightFM(no_components=10, loss="warp", epochs=20, item_alpha=0, user_alpha=0)
recommender = Recommender(model)

In [ ]:
recommender.bulk_fit(train_df)

In [ ]:
recommender.evaluate(test_df, recommend_size=10, filter_interacted=True)